In [1]:
#Install these packages in tensorflow environment before executing the code
#pip install selenium
#pip install webdriver-manager
#pip install selectorlib
#sudo pip3 install keyboard

# The code may not be complete as per the requirements so read the problem statement again and make changes as required

#Provide these inputs
item_name =input("Enter Item Name: ")
email_id=input("Enter Email Id: ")
threshold_price=input("Enter threshold value in dollars: ")

Enter Item Name:  Mobile phones
Enter Email Id:  saumya08gupta@gmail.com
Enter threshold value:  200


In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selectorlib import Extractor
from datetime import datetime
import requests
import json
import time
import pandas as pd

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.amazon.com')
search_box = driver.find_element_by_id('twotabsearchtextbox').send_keys(item_name) #SEarching the item in amazon search window
search_button = driver.find_element_by_id("nav-search-submit-text").click()

driver.implicitly_wait(5)

url=driver.current_url
html = driver.page_source

driver.quit()
titles=[]
time_stamps=[]
prices=[]
e = Extractor.from_yaml_file('selector.yml') # make sure to keep selector.yml file in the same folder as this jupyter notebook
data = e.extract(html)
i=0
if data:
    for product in data['products']:
        titles.append(product['title'])
        time_stamps.append(datetime.now().strftime('%Y-%m-%d %Hh%Mm')) # current timestamp
        if(product['price'] is None): # if price of product is not available on site
            prices.append(0)
        else:
            prices.append(product['price'])
        i+=1
        if(i==5):# Storing only top 5 results
            break
dict={'titles':titles,'timestamps':time_stamps,'price':prices}
alldetails=pd.DataFrame(dict)
print(alldetails)
data = pd.DataFrame(alldetails)
data.to_csv('Amazon_Data.csv') #Save data in excel sheet as csv
print('Data stored in csv file')

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/home/saumya/.wdm/drivers/chromedriver/linux64/87.0.4280.88/chromedriver] found in cache


 
                                              titles         timestamps  \
0  Simple Mobile Samsung Galaxy A01 4G LTE Prepai...  2021-01-24 15h21m   
1  Simple Mobile Samsung Galaxy A20 4G LTE Prepai...  2021-01-24 15h21m   
2  Simple Mobile Samsung Galaxy A01 4G LTE Prepai...  2021-01-24 15h21m   
3  Google Pixel 4a - New Unlocked Android Smartph...  2021-01-24 15h21m   
4  Samsung Galaxy A51 Factory Unlocked Cell Phone...  2021-01-24 15h21m   

     price  
0   $53.39  
1  $144.65  
2   $53.39  
3        0  
4  $399.99  
Data stored in csv file


In [3]:
 #Code to send mail
    
import smtplib, ssl
def sendmail(title):
    port = 587 
    smtp_server = "smtp.gmail.com"
    sender_email = "#######" #Provide email id from which you want to mail 
    # Make sure to provide less secure app access from this email id by going into settings of this provided email id
    receiver_email = email_id
    password = "######" # enter password of email id from which you want to send mails
    message = """\
    Subject: Amazon price below threshold

    Price of the {} titled {} that you wanted is now below threshold value {}.""".format(item_name,title,threshold_price)

    context = ssl.create_default_context()
    with smtplib.SMTP(smtp_server, port) as server:
        server.starttls(context=context)
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, message)
        print('Email sent')

In [4]:
# The continous backend that will check again and again if price of the item falls below the threshold value

import keyboard
while(True):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get('https://www.amazon.com')
    search_box = driver.find_element_by_id('twotabsearchtextbox').send_keys(item_name)
    search_button = driver.find_element_by_id("nav-search-submit-text").click()

    driver.implicitly_wait(5)
    url=driver.current_url
    html = driver.page_source
    driver.quit()
    e = Extractor.from_yaml_file('selector.yml')
    data = e.extract(html)
    new_prices=[]
    if data:
        for product in data['products']:
            if(product['title'] in titles):
                i=titles.index(product['title'])
                # if price of product has changed and is below threshold value now then send mail
                if(prices[i]!=product['price'] and product['price'] is not None and product['price']<threshold_price):
                    sendmail(product['title'])
                if(product['price'] is None): # if price of product is not available on site
                    new_prices.append(0)
                else:
                    new_prices.append(product['price']) #store all new prices
    prices=new_prices # updating prices array
                    
    if keyboard.is_pressed('q'):  #press q to exit the loop
        break


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/home/saumya/.wdm/drivers/chromedriver/linux64/87.0.4280.88/chromedriver] found in cache


ImportError: You must be root to use this library on linux.